In [2]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

## Introdcution 

we are trying to find the gender fo miss calcified names for a data entries made in the customer service at El-Araby Group. the person Data like phone numbers and other confidintial data are removed for privacy reason. 

So in this project I will find a dataset and machine learning model to make a model that correctly classifies the gender of each name. I will do that by using *Random Forest ALgorithm* and then apply the model on the freezed data set to classifiy its gender.


*Objectives*:
- clean the data from anything that may offer corrupt the finiding 
- develop a machine learning algorthim that can predict the gender

### Import  all the Dataframes I will use for this project

In [3]:
freezed_df = pd.read_csv("FreezedNames_final - FreezedNames.csv")


# These two are different dataset that I will merge together and clean them both and finally train the model on
train_df = pd.read_csv("train_data.csv")
train_df2 = pd.read_csv("arabic_names - arabic_names (1).csv")


FileNotFoundError: [Errno 2] No such file or directory: 'FreezedNames_final - FreezedNames.csv'

### Analyze Train_df2

In [ ]:
train_df.sample(10)

,Names,Gender
3118,عبدالو,M
4013,ابومرسي,M
1206,بطاطا,M
1416,جمالات,F
2636,سدراوي,M
5446,وفاء,F
510,عجيب,M
3345,دخليل,M
4981,امواج,F
5994,غلام,M


I already cleaned the data on Excel and SQL 

In [ ]:
train_df.describe()

,Names,Gender
count,6294,6296
unique,6283,2
top,اكرم,M
freq,2,4776


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6296 entries, 0 to 6295
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Names   6294 non-null   object
 1   Gender  6296 non-null   object
dtypes: object(2)
memory usage: 98.5+ KB


In [ ]:
print(train_df["Names"].isna().sum())
print(train_df["Gender"].isna().sum())

2
0


### Remove the Null values

In [ ]:
train_df = train_df[(~train_df["Names"].isna())].reset_index()

train_df["Names"].isna().sum()

0

Do the same cleaning Process on the train_df2

In [ ]:
train_df2.sample(10)

,arabic_name,gender
1062,ستار,M
650,مقبلة,F
892,جساس,M
973,رائف,M
1589,هلال,M
1024,رمزي,M
1243,عزب,M
1214,عبدالنور,M
333,ثناء,F
235,آية,F


In [ ]:
train_df2.describe()

,arabic_name,gender
count,1611,1611
unique,1482,2
top,فرح,M
freq,3,980


In [ ]:
train_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1611 entries, 0 to 1610
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   arabic_name  1611 non-null   object
 1   gender       1611 non-null   object
dtypes: object(2)
memory usage: 25.3+ KB


In [ ]:
print(train_df2["arabic_name"].isna().sum())
print(train_df2["gender"].isna().sum())

0
0


### Merge the Two Datasets into one 

In [ ]:
train_df2 = train_df2.rename(columns = {"arabic_name" : "Names" , 'gender': 'Gender'})

In [ ]:
merged_df = pd.concat([train_df, train_df2], ignore_index=True)

# Print the result
merged_df.tail(10)

,index,Names,Gender
7895,NaN,ولاء الدين,M
7896,NaN,وليد,M
7897,NaN,وليف,M
7898,NaN,وهاب,M
7899,NaN,وهب,M
7900,NaN,وهبة,M
7901,NaN,ياسر,M
7902,NaN,يحيى,M
7903,NaN,يزيد,M
7904,NaN,يعقوب,M


In [ ]:
merged_df.isna().sum()

index     1611
Names        0
Gender       0
dtype: int64

In [ ]:
train_df = merged_df

In [ ]:
index = length[length == longest_name].index[0]
train_df.iloc[index]

index                2310.0
Names     عبدالمعبدعبدالنبي
Gender                    M
Name: 2309, dtype: object

### There are names that have spaces in them 

In [ ]:
def Split(x):
    if(type(x) == float):
        print(x)
    return x.split("\xa0")[-1]

test = train_df["Names"].apply(Split)
train_df["Names"] = pd.DataFrame(test)

In [ ]:
# Assuming 'Names' is the column containing names in your DataFrame
length = train_df["Names"].apply(len)
longest_name = max(length)
index = length.sort_values(ascending=False).index

In [ ]:
# Reorder the rows of the DataFrame using the sorted index
sorted_train_df = train_df.loc[index]

In [ ]:
display(sorted_train_df[sorted_train_df['Names'].apply(len) == 1])

,index,Names,Gender
2344,2345.0,ع,M
315,315.0,ل,M
1923,1924.0,ا,M
1378,1379.0,-,M
4615,4617.0,ء,M
2667,2668.0,ح,M
2391,2392.0,م,M
2166,2167.0,ف,M
2329,2330.0,ض,M
2380,2381.0,ز,M


In [ ]:
sorted_train_df = sorted_train_df[sorted_train_df['Names'].apply(len) != 1]
sorted_train_df.head(10)

,index,Names,Gender
2309,2310.0,عبدالمعبدعبدالنبي,M
3375,3376.0,ناصرعبدالموجود,M
522,522.0,محمدعبدالمقصود,M
4323,4325.0,معبدعبدالوهاب,M
3868,3869.0,معبدعبدالرازق,M
4384,4386.0,محمودعبدالعال,M
3713,3714.0,معبدعبدالمالك,M
4550,4552.0,محمدعبدالجواد,M
1483,1484.0,عامرعبدالمطلب,M
4368,4370.0,السيدمحمدمكين,M


In [ ]:
train_df = sorted_train_df[["Names" , "Gender"]].reset_index(drop=True)

# Prerpocess the freezed_df

In [ ]:
#first upload the dataset for the freezed names and show a smaple of it
freezed_df.head()

,Name
0,​انور ناجى شامخ
1,​وجدي عزت متري
2,• محمود عطيه حسانين
3,1000553724
4,1001146286


there is a problem with the data where there are english and arabic names as well as errors entering the data and finally there are numbers in the name field 

since the data we need to find it all about strings we need to drop the data that has only numbers

In [ ]:
#find the number of numeric cells
numeric_mask = pd.to_numeric(freezed_df['Name'], errors='coerce').notnull()
len(freezed_df[numeric_mask])

1450

In [ ]:
freezed_df.head()

,Name
0,​انور ناجى شامخ
1,​وجدي عزت متري
2,• محمود عطيه حسانين
3,1000553724
4,1001146286


In [ ]:
#remove all the numeric values from the data set
freezed_df = freezed_df[~numeric_mask].reset_index(drop = True)
freezed_df.sample(25, random_state = 42)


,Name
92257,سامه نبيل محمد
196530,ولبيد ربيع رضوان
110790,روفيدا سليمان علي
45255,ايميل اديب تكله
170057,قمر فرغلي حسين
136444,شازلي عبدلله سعد موسي
117546,رفت محمود توفيق
140939,مارسيل شوقى اسكندر
196114,وردشان رمضان محمد شحاته مصطفي
31033,ابراهيم حسن عبدالرحمن رمضان


### 1- remove all the special charaters

In [ ]:
import re

test = freezed_df.copy()
test["Name"] = test["Name"].str.replace(r"[@#$%^&*()_+{}<>?/|•٠1234567890.`]", "")
freezed_df = test

#check for null values
freezed_df.isna().sum()

Name    3
dtype: int64

In [ ]:
freezed_df.dropna(inplace=True)
freezed_df.isna().sum()

Name    0
dtype: int64

### 2 -  here is the data where there are english and arabic names 

In [ ]:
display(freezed_df.iloc[145:165])

,Name
145,LYNN MARIE
146,lمحمد ابراهيم حافظ سالم
147,lمحمد جمال الدين حسين
148,lمحمد حلمى المهدى
149,lمحمد صابر مصطفي
150,lمحمد عبد القادر
151,lمحمد عبداله
152,lمحمد فهمى
153,lمحمد يحيي ابراهيم
154,Lمحمد يحيى دسوقى


In [ ]:
import re
pattern = r'[A-Za-z]'
test = freezed_df.copy()
# test = test.to_dict()
pattern = r'[A-Za-z]'

def remove_english_letters(text):
  if re.search(r'[\u0600-\u06FF]', text):
    return re.sub(pattern, '', text)
  else:
    return text


test["Name"] = test["Name"].apply(remove_english_letters)

    
freezed_names = test
display(test.iloc[145:165])

,Name
145,LYNN MARIE
146,محمد ابراهيم حافظ سالم
147,محمد جمال الدين حسين
148,محمد حلمى المهدى
149,محمد صابر مصطفي
150,محمد عبد القادر
151,محمد عبداله
152,محمد فهمى
153,محمد يحيي ابراهيم
154,محمد يحيى دسوقى


next I am going to make a new column called language where it is going to assign 1 when it has english and 0 for arabic 
to be able to group the data depending on the being written in english or arabic

0: arabic <br>
1: english


In [ ]:
def which_lan(text):
  if re.search(r'[\u0600-\u06FF]', text):
    return 0
  else:
    return 1


lang = []
for row in freezed_df["Name"]:
    lang.append(which_lan(row))
freezed_df = test
freezed_df = freezed_df.assign(lang = lang)
freezed_df.iloc[145:165]

,Name,lang
145,LYNN MARIE,1
146,محمد ابراهيم حافظ سالم,0
147,محمد جمال الدين حسين,0
148,محمد حلمى المهدى,0
149,محمد صابر مصطفي,0
150,محمد عبد القادر,0
151,محمد عبداله,0
152,محمد فهمى,0
153,محمد يحيي ابراهيم,0
154,محمد يحيى دسوقى,0


In [ ]:
def remove_leading_spaces(name):
  return name.lstrip("\s")

freezed_df["Name"] = freezed_df["Name"].apply(remove_leading_spaces)

display(freezed_df.head())

,Name,lang
0,​انور ناجى شامخ,0
1,​وجدي عزت متري,0
2,محمود عطيه حسانين,0
3,سماح محمد عبد الحافظ,0
4,صباح عبد الشافى محمد ح,0


In [ ]:
#there are many entries where it consists of only a space

test = freezed_df.copy()
test = test[~(test["Name"] == " ")]
test = test[~(test["Name"] == "  ")]
freezed_df = test
# for index , name in enumerate(freezed_df["Name"]):
#     if(name == " "):
#        print(index)
#        freezed_df.drop(index , inplace = True)    
display(freezed_df.iloc[1510 :1520])

,Name,lang
1510,nagui nosseir,1
1512,​انطونيو وهبه حكيم,0
1514,الحسينى احمد ابو العزم,0
1515,محمود حسن عبد المجيد,0
1516,ابو الحسن عبد الرحيم الصاوى,0
1517,هشام على فتحى,0
1518,سيد عبد التواب عيد,0
1519,محمد عبدالله,0
1520,أيهاب احمد,0
1521,يهاب علي موافي,0


In [ ]:
def get_first_name(name):
  filtered_array = list(filter(bool, name.split(" ")))
  try:
        return filtered_array[0]
  except IndexError :
    print(name , "sdff", freezed_df.loc[name])
  


first_names =  freezed_df["Name"].apply(get_first_name)

test = freezed_df.copy()
test  = test.assign(first_names = first_names)
freezed_df = test
freezed_df.head()

,Name,lang,first_names
0,​انور ناجى شامخ,0,​انور
1,​وجدي عزت متري,0,​وجدي
2,محمود عطيه حسانين,0,محمود
3,سماح محمد عبد الحافظ,0,سماح
4,صباح عبد الشافى محمد ح,0,صباح


In [ ]:
print(freezed_df["Name"].iloc[1512] == " ") 
freezed_df.iloc[1510: 1520]


False


,Name,lang,first_names
1510,nagui nosseir,1,nagui
1512,​انطونيو وهبه حكيم,0,​انطونيو
1514,الحسينى احمد ابو العزم,0,الحسينى
1515,محمود حسن عبد المجيد,0,محمود
1516,ابو الحسن عبد الرحيم الصاوى,0,ابو
1517,هشام على فتحى,0,هشام
1518,سيد عبد التواب عيد,0,سيد
1519,محمد عبدالله,0,محمد
1520,أيهاب احمد,0,أيهاب
1521,يهاب علي موافي,0,يهاب


In [ ]:
inconsistency_mapper = {
    "أ": "ا",
    "إ": "ا",
    "آ": "ا",
    "ة": "ه",
    "ى": "ي",
    "ـ": "",
    " الدين": "الدين",
    "عبد ": "عبد",
    "ابو ": "ابو",
    "ام ": "ام",
}



### Make a table for english and arabic written names

In [ ]:
arabic_names = freezed_df[freezed_df["lang"] ==0]
english_names = freezed_df[freezed_df["lang"] ==1]


In [ ]:
for k, v in inconsistency_mapper.items():
    arabic_names["first_names"] = arabic_names["first_names"].str.replace(k, v)

In [ ]:
gourped_arabic = arabic_names.groupby("first_names").size().to_frame(name="count").reset_index()

gourped_arabic = gourped_arabic.sort_values(by=["count"], ascending=False)
gourped_arabic.head(10)

,first_names,count
30220,نادي,3248
1423,اسلام,2664
26666,محمد,1077
30733,نجاه,1057
24046,قمر,967
9774,جمال,933
4863,المهندس,810
659,احمد,664
19819,ع,637
6578,ايميل,616


In [ ]:
Freezed_arabic_first = arabic_names["first_names"]
Freezed_english_first = arabic_names["first_names"]

In [ ]:
test_freezed = arabic_names["first_names"]
test_freezed[0]

'\u200bانور'

In [ ]:
import re

test = freezed_df.copy()
test_freezed = test_freezed.str.replace(r"[@#$%^&*()_+{}<>?/|•٠1234567890.` ِ\u200bc ]", "")
#check for null values
#test_freezed = test_freezed.replace("" , np.nan).dropna(inplace = True)
#test_freezed.replace(" " , np.nan).dropna(inplace = True)

test_freezed = test_freezed.replace("" , np.nan).dropna()

any(element == "" for element in test_freezed)

False

In [ ]:
Freezed_arabic_first = list(test_freezed)
Freezed_arabic_first[0:5]

['انور', 'وجدي', 'محمود', 'سماح', 'صباح']

In [ ]:
Freezed_names_series = pd.Series(Freezed_arabic_first)
Freezed_arabic_length = Freezed_names_series.apply(len)
Freezed_arabic_length = list(Freezed_arabic_length)
#pd.Series(Freezed_arabic_first)[pd.Series(Freezed_arabic_first) == max(Freezed_arabic_length)]
index_of_value = Freezed_arabic_length.index(max(Freezed_arabic_length))
Freezed_arabic_first[index_of_value]

'العالميهلتجارهالجملهوالتجزئهللاجهزه'

In [ ]:
mask = (Freezed_names_series.str.len() <=17 ) & (Freezed_names_series.str.len() >1 )
Freezed_names_series = Freezed_names_series[mask]
Freezed_arabic_length = Freezed_names_series.apply(len)
Freezed_arabic_length = list(Freezed_arabic_length)
index_of_value = Freezed_arabic_length.index(max(Freezed_arabic_length))

In [ ]:
print(sorted(Freezed_arabic_length, reverse=True)[0:20])

[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]


In [ ]:
len(Freezed_arabic_length) , len(Freezed_names_df)

(177994, 177994)

In [ ]:
Freezed_arabic_length = pd.Series(Freezed_arabic_length)
index2 = Freezed_arabic_length.sort_values(ascending=False).index
Freezed_names_df = pd.DataFrame(Freezed_names_series)
Freezed_names_df = Freezed_names_df.iloc[index2]
Freezed_arabic_first = Freezed_names_df[0]
Freezed_arabic_first

105749    عمرابراهيمالخواجه
79458     صابرعبدالناصرحيدر
68048     سعدسعيدمحمدالحلاج
176007    ناديه عزيز ساويرس
138803    محمودمحمدمحمداحمد
                ...        
84995                    سر
76768                    طه
101236                   فا
61672                    حر
115662                   ضا
Name: 0, Length: 177994, dtype: object

# Train the machine learning algorithm for arabic names

### Naive Bayes Model

In [ ]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
train_data = train_df
train_data

,Names,Gender
0,عبدالمعبدعبدالنبي,M
1,محمدعبدالمقصود,M
2,ناصرعبدالموجود,M
3,عامرعبدالمطلب,M
4,محمدعبدالحميد,M
...,...,...
6274,مل,M
6275,خص,F
6276,يم,M
6277,ام,F


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import nltk

nltk.download("stopwords")  # Download the stopwords corpus

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_data["Names"], train_data["Gender"], test_size=0.2, random_state=42)

# Create a TfidfVectorizer object and fit it to the training data
arabic_stop_words = set(nltk.corpus.stopwords.words("arabic"))
vectorizer = TfidfVectorizer(stop_words=list(arabic_stop_words))  # Convert set to list
X_train_tfidf = vectorizer.fit_transform(X_train)

# Create a MultinomialNB classifier and fit it to the training data
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

# Predict the gender of the names in the test set
y_pred = clf.predict(vectorizer.transform(X_test))

# Calculate the accuracy of the model
accuracy = (y_pred == y_test).mean()

# Print the accuracy of the model
print("Accuracy:", accuracy)


Accuracy: 0.7611464968152867


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Encoding the values to be numrical instead of string for the model 

In [ ]:
# Assuming your DataFrame is named df and it has a column named 'word'
df = train_data

# Split the 'word' column into individual letters and create new columns
df = df['Names'].apply(lambda x: pd.Series(list(x)))

# Rename the columns to have meaningful names if needed
df.columns = [f'letter_{i+1}' for i in range(df.shape[1])]

# Display the resulting DataFrame
display(df)

,letter_1,letter_2,letter_3,letter_4,letter_5,letter_6,letter_7,letter_8,letter_9,letter_10,letter_11,letter_12,letter_13,letter_14,letter_15,letter_16,letter_17
0,ع,ب,د,ا,ل,م,ع,ب,د,ع,ب,د,ا,ل,ن,ب,ي
1,م,ح,م,د,ع,ب,د,ا,ل,م,ق,ص,و,د,NaN,NaN,NaN
2,ن,ا,ص,ر,ع,ب,د,ا,ل,م,و,ج,و,د,NaN,NaN,NaN
3,ع,ا,م,ر,ع,ب,د,ا,ل,م,ط,ل,ب,NaN,NaN,NaN,NaN
4,م,ح,م,د,ع,ب,د,ا,ل,ح,م,ي,د,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274,م,ل,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6275,خ,ص,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6276,ي,م,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6277,ا,م,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# The dictionray for the mapping 
arabic_alphabet_mapping = {
    'ا': 1, 'ب': 2, 'ت': 3, 'ث': 4, 'ج': 5, 'ح': 6, 'خ': 7, 'د': 8, 'ذ': 9, 'ر': 10,
    'ز': 11, 'س': 12, 'ش': 13, 'ص': 14, 'ض': 15, 'ط': 16, 'ظ': 17, 'ع': 18, 'غ': 19,
    'ف': 20, 'ق': 21, 'ك': 22, 'ل': 23, 'م': 24, 'ن': 25, 'ه': 26, 'و': 27, 'ي': 28
}
x = df
columns_to_encode = x.columns

test = x.copy()
# Iterate through each column and apply mapping
for column in columns_to_encode:
    test[column] = test[column].map(arabic_alphabet_mapping)


,letter_1,letter_2,letter_3,letter_4,letter_5,letter_6,letter_7,letter_8,letter_9,letter_10,letter_11,letter_12,letter_13,letter_14,letter_15,letter_16,letter_17
0,18.0,2.0,8.0,1.0,23.0,24.0,18.0,2.0,8.0,18.0,2.0,8.0,1.0,23.0,25.0,2.0,28.0
1,24.0,6.0,24.0,8.0,18.0,2.0,8.0,1.0,23.0,24.0,21.0,14.0,27.0,8.0,NaN,NaN,NaN
2,25.0,1.0,14.0,10.0,18.0,2.0,8.0,1.0,23.0,24.0,27.0,5.0,27.0,8.0,NaN,NaN,NaN
3,18.0,1.0,24.0,10.0,18.0,2.0,8.0,1.0,23.0,24.0,16.0,23.0,2.0,NaN,NaN,NaN,NaN
4,24.0,6.0,24.0,8.0,18.0,2.0,8.0,1.0,23.0,6.0,24.0,28.0,8.0,NaN,NaN,NaN,NaN


In [ ]:
def decode_pd(df):
    # Split the 'word' column into individual letters and create new columns
    if(type(df)== type(pd.DataFrame())):    
        df = df["Names"].apply(lambda x: pd.Series(list(x)))
    else:
        df = pd.Series(df).apply(lambda x: pd.Series(list(x)))
    # Rename the columns to have meaningful names if needed
    df.columns = [f'letter_{i+1}' for i in range(df.shape[1])]
    # The dictionray for the mapping 
    arabic_alphabet_mapping = {
        'ا': 1, 'ب': 2, 'ت': 3, 'ث': 4, 'ج': 5, 'ح': 6, 'خ': 7, 'د': 8, 'ذ': 9, 'ر': 10,
        'ز': 11, 'س': 12, 'ش': 13, 'ص': 14, 'ض': 15, 'ط': 16, 'ظ': 17, 'ع': 18, 'غ': 19,
        'ف': 20, 'ق': 21, 'ك': 22, 'ل': 23, 'م': 24, 'ن': 25, 'ه': 26, 'و': 27, 'ي': 28
    }
    x = df
    columns_to_encode = x.columns

    test = x.copy()
    # Iterate through each column and apply mapping
    for column in columns_to_encode:
        test[column] = test[column].map(arabic_alphabet_mapping)
    return test
    # Now your DataFrame is encoded with numerical values

In [ ]:
test = decode_pd(train_data)
test

,letter_1,letter_2,letter_3,letter_4,letter_5,letter_6,letter_7,letter_8,letter_9,letter_10,letter_11,letter_12,letter_13,letter_14,letter_15,letter_16,letter_17
0,18.0,2.0,8.0,1.0,23.0,24.0,18.0,2.0,8.0,18.0,2.0,8.0,1.0,23.0,25.0,2.0,28.0
1,24.0,6.0,24.0,8.0,18.0,2.0,8.0,1.0,23.0,24.0,21.0,14.0,27.0,8.0,NaN,NaN,NaN
2,25.0,1.0,14.0,10.0,18.0,2.0,8.0,1.0,23.0,24.0,27.0,5.0,27.0,8.0,NaN,NaN,NaN
3,18.0,1.0,24.0,10.0,18.0,2.0,8.0,1.0,23.0,24.0,16.0,23.0,2.0,NaN,NaN,NaN,NaN
4,24.0,6.0,24.0,8.0,18.0,2.0,8.0,1.0,23.0,6.0,24.0,28.0,8.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274,24.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6275,7.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6276,28.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6277,1.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test2 = decode_pd(Freezed_arabic_first)
test2

,letter_1,letter_2,letter_3,letter_4,letter_5,letter_6,letter_7,letter_8,letter_9,letter_10,letter_11,letter_12,letter_13,letter_14,letter_15,letter_16,letter_17
105749,18.0,24.0,10.0,1.0,2.0,10.0,1.0,26.0,28.0,24.0,1.0,23.0,7.0,27.0,1.0,5.0,26.0
79458,14.0,1.0,2.0,10.0,18.0,2.0,8.0,1.0,23.0,25.0,1.0,14.0,10.0,6.0,28.0,8.0,10.0
68048,12.0,18.0,8.0,12.0,18.0,28.0,8.0,24.0,6.0,24.0,8.0,1.0,23.0,6.0,23.0,1.0,5.0
176007,25.0,1.0,8.0,28.0,26.0,NaN,18.0,11.0,28.0,11.0,NaN,12.0,1.0,27.0,28.0,10.0,12.0
138803,24.0,6.0,24.0,27.0,8.0,24.0,6.0,24.0,8.0,24.0,6.0,24.0,8.0,1.0,6.0,24.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84995,12.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76768,16.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101236,20.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61672,6.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Freezed_arabic_decoded = test2

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(train_data.Gender)
x = test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Instantiate the imputer with mean strategy
imputer = SimpleImputer(strategy='mean')

# Fit and transform the imputer on training data
x_train_imputed = imputer.fit_transform(x_train)

# Transform the test data using the trained imputer
x_test_imputed = imputer.transform(x_test)

# Now, you can train your RandomForestClassifier
rf_classifier.fit(x_train_imputed, y_train)
# Evaluate the model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8702229299363057


In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier


hist_gradient_boosting_classifier = HistGradientBoostingClassifier(random_state=42)

# Train the classifier on the training data
hist_gradient_boosting_classifier.fit(x_train, y_train)

# Make predictions on the test data
y_pred = hist_gradient_boosting_classifier.predict(x_test)

# Evaluate the model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")



Accuracy: 0.8702229299363057


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    MultinomialNB()]

for name, clf in zip(names, classifiers):
    print(name,   clf)
    clf.fit(x_train_imputed, y_train)
    score = clf.score(x_test_imputed, y_test)
    print("Validation Accuracy",score*100,"%")

Nearest Neighbors KNeighborsClassifier(n_neighbors=3)
Validation Accuracy 76.671974522293 %
Linear SVM SVC(C=0.025, kernel='linear')
Validation Accuracy 76.11464968152866 %
RBF SVM SVC(C=1, gamma=2)
Validation Accuracy 76.35350318471338 %
Decision Tree DecisionTreeClassifier(max_depth=5)
Validation Accuracy 81.21019108280255 %
Random Forest RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10)
Validation Accuracy 76.99044585987261 %
Neural Net MLPClassifier(alpha=1, max_iter=1000)
Validation Accuracy 77.22929936305732 %
AdaBoost AdaBoostClassifier()
Validation Accuracy 82.96178343949045 %
Naive Bayes MultinomialNB()
Validation Accuracy 64.49044585987261 %


In [ ]:
clf = hist_gradient_boosting_classifier.fit(x_train_imputed, y_train)
score = clf.score(x_test_imputed, y_test)
print("Validation Accuracy",score*100,"%")

Validation Accuracy 87.10191082802548 %


In [ ]:
predict = clf.predict(x_test_imputed)

results_df = pd.DataFrame({
    'Actual': X_test,
    'Predicted': predict
})

# Display the DataFrame
pd.DataFrame(results_df).head()

,Actual,Predicted
4387,مدحه,0
5296,طالع,1
2873,سلموس,1
6075,بوش,1
3167,نيفين,0


In [ ]:
predict_freezed = clf.predict(x_test_imputed)

results_df = pd.DataFrame({
    'Actual': X_test,
    'Predicted': predict
})

# Display the DataFrame
pd.DataFrame(results_df).head()

In [ ]:

# Instantiate the imputer with mean strategy
imputer = SimpleImputer(strategy='mean')

# Fit and transform the imputer on Freezed_arabic_first
Freezed_arabic_decoded = imputer.fit_transform(Freezed_arabic_decoded)
predict_freezed = clf.predict(test2)

results_df = pd.DataFrame({
    'Actual': X_test,
    'Predicted': predict
})

# Display the DataFrame
pd.DataFrame(results_df).head()

,Actual,Predicted
4387,مدحه,0
5296,طالع,1
2873,سلموس,1
6075,بوش,1
3167,نيفين,0


In [ ]:
test3 = decode_pd(["نوررررررررررررررر", "نعمه"])

predict_freezed = clf.predict(test3)
predict_freezed

array([1, 0])

## Conclusion 

The model that has the most accuracy is random forest tree with ambutation and also the HistGradientBoostingClassifier 
With accuracy 87%